In [2]:
import pandas as pd
import os
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
path = os.getenv("path_to_data")
path_master = os.getenv("path_to_master")

In [4]:
#!/usr/bin/env python3
"""
Playwright crawler (Notebook‑compatible) for LUBW Fließgewässer‑Dashboard.

*   wählt Kreise **Esslingen** und **Stuttgart, Landeshauptstadt**
*   setzt das Datumsfeld auf **"2024 - <aktuelles Jahr>"**
*   hovert über den Spaltenkopf »Parameter«, klickt Export → *CSV‑Datei*
*   lässt den Browser sichtbar laufen (`headless=False`), damit du alles siehst

Erstinstallation:
    pip install playwright
    playwright install

Notebook‑Aufruf:
    await main(headless=False)
"""
import asyncio
from datetime import datetime
from pathlib import Path
from playwright.async_api import async_playwright

URL = (
    "https://umweltdaten.lubw.baden-wuerttemberg.de/repositories/"
    "wasser_gewaesserguete,yUm3pYRGyaP7hFRow7Rf/workbooks/"
    "Fliessgewaesserguete,8SMZrw9xObs2ChqTSHk1/worksheets/"
    "Daten-der-Online-Messstationen,XucPlXdlipPF63XPBcOz"
    "?workbookHash=d7e9GgWwzUgJQK5oRyJbAwEQa-fEcb54KVJ-nA2GRzL45K5G"
    "&embeddingTargetId=chemie-messstellen"
)

KREISE = ["Esslingen", "Stuttgart, Landeshauptstadt"]

async def choose_kreise(page):
    """Wählt die gewünschten Kreise im Select2-Filter aus."""
    print("🔎 Wähle Kreise:", KREISE)

    # Warte auf das Select2-Suchfeld (sichtbar & interaktiv)
    await page.wait_for_selector("input.select2-search__field", timeout=15000)

    for kreis in KREISE:
        # Klicke in das Eingabefeld (damit das Dropdown erscheint)
        await page.click("input.select2-search__field")

        # Tippe den Namen des Kreis
        await page.fill("input.select2-search__field", kreis)

        # Warte und klicke auf das Dropdown-Ergebnis
        await page.wait_for_selector('.select2-results__option', timeout=5000)
        await page.locator('.select2-results__option', has_text=kreis).click()

        # Kurze Pause zur Stabilität
        await page.wait_for_timeout(400)

    print("✅ Kreise erfolgreich gewählt.")

async def set_datum_filter(page):
    """Setzt das Datumsfeld direkt per Texteingabe."""
    current_year = datetime.now().year
    date_range = f"2024 - {current_year}"

    # Textarea mit Placeholder "yyyy - yyyy"
    date_input = page.locator("textarea.d-condition-date-picker--input.form-control[placeholder='yyyy - yyyy']").first
    await date_input.scroll_into_view_if_needed()
    await date_input.wait_for(state="visible", timeout=10000)
    await date_input.fill(date_range)
    await date_input.press("Enter")
    print(f"✔ Datum gesetzt: {date_range}")

async def export_csv(page):
    """Hovert über Spaltenkopf 'Parameter' und exportiert CSV."""
    # 1) Spaltenkopf „Parameter“ in Sicht bringen & hovern
    header = page.locator("th:has-text('Parameter')").first
    await header.scroll_into_view_if_needed()
    await header.hover()
    await page.wait_for_timeout(500)

    # 2) Auf das Export-Icon in der Toolbar warten und klicken
    export_btn = page.locator("button[aria-label='Exportieren']").first
    await export_btn.wait_for(state="visible", timeout=4000)
    await export_btn.click()

    # 3) Menüpunkt „CSV-Datei“ abwarten & klicken
    csv_entry = page.locator("role=menuitem >> text=CSV-Datei").first
    await csv_entry.wait_for(state="visible", timeout=4000)
    async with page.expect_download() as dl_info:
        await csv_entry.click()

    # 4) Download speichern (Dateiname anpassen, falls gewünscht)
    download = await dl_info.value
    await download.save_as(f"{path}/lubw_download_latest.csv")
    print("✔ CSV erfolgreich gespeichert")


async def main(headless=False):
    async with async_playwright() as pw:
        browser = await pw.chromium.launch(headless=headless)
        context = await browser.new_context()
        page = await context.new_page()

        await page.goto(URL, wait_until="domcontentloaded")
        await page.wait_for_timeout(3000)  # Dashboard initialisieren

        # Kreise wählen
        await choose_kreise(page)
        await page.wait_for_timeout(2000)

        # Datum setzen
        await set_datum_filter(page)
        await page.wait_for_timeout(1500)

        # Export
        await export_csv(page)

        print("✅ Ablauf abgeschlossen – falls ein nativer Save‑Dialog erscheint, bitte manuell speichern.")
        await browser.close()


# Für Notebook:
await main(headless=False)


🔎 Wähle Kreise: ['Esslingen', 'Stuttgart, Landeshauptstadt']
✅ Kreise erfolgreich gewählt.
✔ Datum gesetzt: 2024 - 2025
✔ CSV erfolgreich gespeichert
✅ Ablauf abgeschlossen – falls ein nativer Save‑Dialog erscheint, bitte manuell speichern.
